# DATA CLEANING & FEATURE ENGINEERING- NOUAR Alexandre & DURA Théo

The goal of this notebook is see how to clean the data and to do the feature engineering

In [6]:
import pandas as pd
import numpy as np

In [117]:
test_df = pd.read_csv('../data/raw/raw_test_data.csv')
train_df = pd.read_csv('../data/raw/raw_train_data.csv')

In [118]:
#x_train = train_df.drop(['TARGET'], axis=1)
#y_train = train_df['TARGET']

#x_test = test_df

In [128]:
print(train_df.shape, test_df.shape)

(307511, 122) (25820, 101)


## Dealing with missing values :

Since there is a lot of missing values, we can't just drop them all as we will go from 300 000+ row to only 30 000.

First we will drop the columns with more than 60% missing values:

In [29]:
def missing_values_columns(df):
        # count the total number of missing value in the dataframe
        missing = df.isnull().sum()

        # Makes it a percentage
        percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        table = pd.concat([missing, percent], axis=1)
        
        # Rename the columns
        table_rename = table.rename(columns = {0: 'Number of missing values',1: '% of Total Values'})
        
        return table_rename[table_rename['% of Total Values'] > 59].index

In [132]:
train_todrop = missing_values_columns(train_df)
test_todrop = missing_values_columns(test_df)

In [133]:
if len(train_todrop)>len(test_todrop):
    todrop = train_todrop;
else:
    todrop = test_todrop

In [134]:
train_df.drop(todrop, axis=1, inplace=True)
test_df.drop(todrop, axis=1, inplace=True)

In [135]:
print(train_df.shape, test_df.shape)

(307511, 102) (25820, 101)


Now we will drop the rows that contains more than 80% of missing values as they are irrelevent

In [136]:
train_df.dropna(axis = 0, how = 'any', thresh = int(train_df.shape[1]*0.8),inplace=True)
test_df.dropna(axis = 0, how = 'any', thresh = int(test_df.shape[1]*0.8),inplace=True)

In [137]:
print(train_df.shape, test_df.shape)

(157036, 102) (25820, 101)


We are gonna replace every missing values in the qualitative columns by the mode:

In [142]:
qualitative_c = test_df.select_dtypes(include=[object]).columns

In [201]:
for col in qualitative_c:
    train_df[col] = train_df[col].fillna(train_df[col].mode(dropna=True)[0])
    test_df[col] = test_df[col].fillna(test_df[col].mode(dropna=True)[0])

Then every missing values in quantitative columns will be replaced by the median:

In [145]:
quantitative_c = test_df.select_dtypes(include=[int,float]).columns

In [146]:
for col in quantitative_c:
    train_df[col] = train_df[col].fillna(train_df[col].median())
    test_df[col] = test_df[col].fillna(test_df[col].median())

In [206]:
print(train_df.columns[train_df.isna().any()].tolist(), test_df.columns[test_df.isna().any()].tolist())

[] []


## Feature Engineering :